Imports
-------

In [1]:
import os

import matplotlib
import numpy as np
import requests
from matplotlib import pyplot as plt
from matplotlib.pyplot import subplots

from genericdlmodel import Model

%matplotlib widget

Download data
-------------

In [2]:
# headers = {
#    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:10.0) Gecko/20100101 Firefox/10.0"
# }
request_opts = {"params": {"raw": "true"}}

In [3]:
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

base_url = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/"
fname = "mnist.npz"
fpath = os.path.join(data_dir, fname)
if not os.path.exists(fpath):
    print("Downloading file: " + fname)
    resp = requests.get(base_url + fname, stream=True, **request_opts)
    resp.raise_for_status()
    with open(fpath, "wb") as fh:
        for chunk in resp.iter_content(chunk_size=128):
            fh.write(chunk)

In [4]:
data_file = np.load(fpath)
x_train = data_file["x_train"]
x_test = data_file["x_test"]
y_train = data_file["y_train"]
y_test = data_file["y_test"]
data_file.close()

Data preprocessing
------------------

In [5]:
x_train = x_train.reshape(len(x_train), 28 * 28)
x_test = x_test.reshape(len(x_test), 28 * 28)

In [6]:
sample_size = int(len(x_train))
test_split = 5 * sample_size // 6
seed = 1234
rng = np.random.default_rng(seed=seed)
index = rng.choice(len(x_train), sample_size)
train_sample_X = x_train[index[:test_split]]
val_sample_X = x_train[index[test_split:]]
train_sample_y = y_train[index[:test_split]]
val_sample_y = y_train[index[test_split:]]

In [7]:
train_sample_X = train_sample_X * 1.0 / 255
val_sample_X = val_sample_X * 1.0 / 255
test_sample_X = x_test * 1.0 / 255

In [8]:
def one_hot_encoding(labels, dim=10):
    one_hot_labels = labels[..., None] == np.arange(dim)[None]
    return one_hot_labels.astype(np.float64)

In [9]:
train_sample_labels = one_hot_encoding(train_sample_y)
val_sample_labels = one_hot_encoding(val_sample_y)
test_sample_labels = one_hot_encoding(y_test)

Initializing hyperparameters
----------------------------

In [13]:
learning_rate = 0.7
epochs = 20
pixels_per_image = 28 * 28
num_labels = 10
batch_size = 200
dropout = 0
hidden_layer_sizes = [200, 50]
update_rule = "svd_lowrank"
update_args = {"rank": 10, "q": 10, "niter": 2}

Running the models
------------------

In [14]:
model = Model(
    rng=rng,
    training_data_X=train_sample_X,
    training_data_y=train_sample_labels,
    val_data_X=val_sample_X,
    val_data_y=val_sample_labels,
    objective_function="categoricalcrossentropy",
    learning_rate=learning_rate,
    epochs=epochs,
    batch_size=batch_size,
)
for output_size in hidden_layer_sizes:
    model.add_layer(
        output_size=output_size,
        func_name="relu",
        dropout=dropout,
        update_rule=update_rule,
        update_args=update_args,
    )
model.add_final_layer()

In [15]:
%%timeit
model.run()

Epoch: 0
  Training loss:          19.565
  Training accuracy:      0.660
  Validation loss:        23.304
  Validation accuracy:    0.799

Epoch: 1
  Training loss:          19.795
  Training accuracy:      0.830
  Validation loss:        23.341
  Validation accuracy:    0.854



KeyboardInterrupt: 

TODO
----
1. test parallelization
2. compare times between rank one update and SVD
3. compare number of iterations for rank one updates
4. test taking svd of svd_lowrank
5. pytorch
6. test learning rate
7. 